## Download ERA5 data  
To compare the forecast data with the real observations, use reanalysis data. Therefore download these data from Copernicus Climate Store.

In [25]:
import cdsapi
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

In [2]:
c = cdsapi.Client()

c.retrieve(
    'reanalysis-era5-single-levels',
    {
        'product_type': 'reanalysis',
        'format': 'netcdf',
        'variable': 'total_precipitation',
        'year': '2024',
        'month': '05',
        'day': [
            '18', '19',
        ],
        'time': [
            '00:00', '06:00', '12:00',
            '18:00',
        ],
    },
    'era5_18_19may.nc')

2024-07-11 07:18:57,803 INFO Welcome to the CDS
2024-07-11 07:18:57,804 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-07-11 07:18:57,934 INFO Request is queued
2024-07-11 07:18:58,971 INFO Request is running
2024-07-11 07:19:06,204 INFO Request is completed
2024-07-11 07:19:06,206 INFO Downloading https://download-0003-clone.copernicus-climate.eu/cache-compute-0003/cache/data5/adaptor.mars.internal-1720682342.3137634-26924-16-260bebec-7c2a-4e33-b039-8e43d3522ab1.nc to era5_18_19may.nc (15.9M)
2024-07-11 07:19:07,502 INFO Download rate 12.2M/s  


Result(content_length=16621584,content_type=application/x-netcdf,location=https://download-0003-clone.copernicus-climate.eu/cache-compute-0003/cache/data5/adaptor.mars.internal-1720682342.3137634-26924-16-260bebec-7c2a-4e33-b039-8e43d3522ab1.nc)

In [26]:
dt=xr.open_dataset('ESDP_project/era5_18_19may.nc')

In [32]:
dt.tp

<xarray.DataArray 'tp' (time: 8, latitude: 721, longitude: 1440)> Size: 66MB
array([[[ 5.822924e-07,  5.822924e-07, ...,  5.822924e-07,  5.822924e-07],
        [-3.469447e-18, -3.469447e-18, ..., -3.469447e-18, -3.469447e-18],
        ...,
        [-3.469447e-18, -3.469447e-18, ..., -3.469447e-18, -3.469447e-18],
        [-3.469447e-18, -3.469447e-18, ..., -3.469447e-18, -3.469447e-18]],

       [[ 1.164585e-06,  1.164585e-06, ...,  1.164585e-06,  1.164585e-06],
        [-3.469447e-18, -3.469447e-18, ..., -3.469447e-18, -3.469447e-18],
        ...,
        [-3.469447e-18, -3.469447e-18, ..., -3.469447e-18, -3.469447e-18],
        [-3.469447e-18, -3.469447e-18, ..., -3.469447e-18, -3.469447e-18]],

       ...,

       [[ 4.076047e-06,  4.076047e-06, ...,  4.076047e-06,  4.076047e-06],
        [ 1.164585e-06,  1.164585e-06, ...,  1.164585e-06,  1.164585e-06],
        ...,
        [ 1.164585e-06,  1.164585e-06, ...,  1.164585e-06,  1.164585e-06],
        [ 1.746877e-06,  1.746877e-06, ...,  1.746877e-06,  1.746877e-06]],

       [[ 5.822924e-07,  5.822924e-07, ...,  5.822924e-07,  5.822924e-07],
        [-3.469447e-18, -3.469447e-18, ..., -3.469447e-18, -3.469447e-18],
        ...,
        [ 2.911462e-06,  2.911462e-06, ...,  2.911462e-06,  2.911462e-06],
        [ 2.911462e-06,  2.911462e-06, ...,  2.911462e-06,  2.911462e-06]]])
Coordinates:
  * longitude  (longitude) float32 6kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
  * latitude   (latitude) float32 3kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * time       (time) datetime64[ns] 64B 2024-05-18 ... 2024-05-19T18:00:00
Attributes:
    units:      m
    long_name:  Total precipitation

Compare ERA5 with Forecast:  
regrid ERA5 on same grid  
calculate difference between them  
mean & std etc

## Convert Units  
The ERA5 Data for total precipitation has the unit [m]. To compare it to the forecasst data, we have to convert it into [kg/m^2]. Therefore, just multplicate the values of ERA5 * 1000.

In [33]:
# convert the values to [kg/m^2]
dt['tp'] = dt['tp'] *1000 

In [34]:
# change the metadata for correct documentation in the new nc-file
dt['tp'].attrs['units'] = 'kg m**-2'

In [35]:
# save into a new netcdf file
output_file = 'era5_new_units.nc'
dt.to_netcdf(output_file)

Now lets check, if it worked correctly.

In [36]:
df = xr.open_dataset(output_file)

In [40]:
df

<xarray.Dataset> Size: 66MB
Dimensions:    (longitude: 1440, latitude: 721, time: 8)
Coordinates:
  * longitude  (longitude) float32 6kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
  * latitude   (latitude) float32 3kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * time       (time) datetime64[ns] 64B 2024-05-18 ... 2024-05-19T18:00:00
Data variables:
    tp         (time, latitude, longitude) float64 66MB 0.0005823 ... 0.002911
Attributes:
    Conventions:  CF-1.6
    history:      2024-07-11 07:19:02 GMT by grib_to_netcdf-2.28.1: /opt/ecmw...

We can see, that tp has now the correct units to compare it with the values from the forecast data.